In [2]:
import uuid
import random
import numpy as np

In [3]:
from tax_calculation import get_taxable_income, calculate_final_tax

In [4]:
from sqlalchemy import create_engine, Column, String, Integer, DateTime
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base

In [5]:
DB_URL = (
    "postgresql://test:must_be_eight_characters"
    + "@example-1.cluster-cculi2axzscc.us-east-1.rds.amazonaws.com:5432/test"
)

engine = create_engine(DB_URL)

# Create a session to interact with the database
Session = sessionmaker(bind=engine)
session = Session()

In [6]:
# Create a base class for declarative models
Base = declarative_base()


# Define the User model
class User(Base):
    __tablename__ = "users"

    id = Column(String, primary_key=True)
    name = Column(String)
    password = Column(String)
    gender = Column(String)
    date_of_birth = Column(DateTime)


# Define the TaxDetails model
class TaxDetails(Base):
    __tablename__ = "tax_details"

    tax_id = Column(String, primary_key=True)
    user_id = Column(String)
    year = Column(Integer)
    income = Column(Integer)
    taxable_income = Column(Integer)
    location = Column(String)
    tax_amount = Column(Integer)

In [13]:
for _ in range(1000):
    user_year = random.randint(1960, 2000)
    new_user = User(
        id=str(uuid.uuid4()),
        name="User Name",
        password="Password",
        gender=random.choice(["male", "female"]),
        date_of_birth=str(user_year) + "-01-01",
    )

    session.add(new_user)

    user_base_income = random.randint(100000, 1000000)
    location = random.choice(["dhaka", "chittagong", "city", "non_city"])

    for year in range(2015, 2022):
        income_in_year = user_base_income + random.randint(10000, 100000)
        new_tax_details = TaxDetails(
            tax_id=str(uuid.uuid4()),
            user_id=new_user.id,
            year=year,
            income=income_in_year,
            taxable_income=get_taxable_income(
                income_in_year, new_user.gender, year - user_year + 1
            ),
            location=location,
            tax_amount=calculate_final_tax(
                income_in_year, new_user.gender, year - user_year + 1, location
            ),
        )

        session.add(new_tax_details)

    session.commit()
# takes 18 min to run